In [2]:
from utils_phys import *
from utils_INN import * 
import random
import matplotlib.pyplot as plt
import tqdm as tq

TORCH DEVICE: cuda
TORCH VERSION: 1.9.1
TORCH DEVICE: cuda


In [3]:
# Lattice Theory
L = 14
lattice_shape = (L,L)

#Model setup

n_layers = 18
hidden_sizes = [12 ,12, 8]
kernel_size = 3
layers = make_phi4_affine_layers(lattice_shape=lattice_shape, n_layers=n_layers, 
    hidden_sizes=hidden_sizes, kernel_size=kernel_size)


# Training
base_lr = .001
optimizer = torch.optim.Adam(layers.parameters(), lr=base_lr)

In [ ]:
# Training
base_lr = .0008
optimizer = torch.optim.Adam(layers.parameters(), lr=base_lr)

print_freq = 2
#Training setup
N_era = 101
N_epoch = 200
batch_size = 64
history = {
'loss' : [],
'logp' : [],
'logq' : [],
'ess' : []
}


#Configuration setup
lam_cur = 0.02
kappa = [0.23, 0.25, 0.255, 0.26, 0.265, 0.27, 0.275, 0.28, 0.285, 0.29]


#define current model with current prior
prior = SimpleNormal(torch.zeros(lattice_shape), torch.ones(lattice_shape))
model = {'layers': layers, 'prior': prior}
trange = tq.trange(N_epoch*N_era)

#Training scheme
for i in trange:
    kappa_cur_ind = np.random.randint(0, len(kappa))
    kappa_cur = kappa[kappa_cur_ind]
    
    
    
    action_cur = ScalarPhi4Action(kappa=kappa_cur, lam=lam_cur)

    #conditioning features
    x_cond = torch.ones((batch_size, L, L)) * kappa_cur * 10

    train_step(model, action_cur, calc_dkl, optimizer, history, batch_size, x_cond)
    trange.set_postfix()
    

## Sampling


In [5]:
def record_flow_mc(
        model, action_fn,
        n_batches: int, batch_size: int, nth: int, conditioning):
    layers, prior = model['layers'], model['prior']
    
    phi = []
    accepted = []
    mag = []
    with torch.no_grad():
        last_phi, last_logq = apply_flow_to_prior(prior, layers, conditioning, batch_size = batch_size)
        last_logp = -action_fn(last_phi)
 
        trange = tq.trange(n_batches)
        for i in trange:
            new_phi, new_logq = apply_flow_to_prior(prior, layers, conditioning, batch_size = batch_size)
            new_logp = -action_fn(new_phi)
            
            p_accept = torch.exp((new_logp - new_logq) - (last_logp - last_logq))
            accept = torch.rand(p_accept.shape) < p_accept
            accepted.append(grab(accept))
            a = accept.view(-1,1,1).repeat(1,*new_phi.shape[1:])

            last_phi = a * new_phi + ~a * last_phi
            last_logq = accept * new_logq + ~accept * last_logq
            last_logp = accept * new_logp + ~accept * last_logp

            M = get_mag(last_phi)
            ind = np.where(M < 0)
            M[ind] = M[ind] * -1 
            mag.append(M)
            if (i+1) % nth == 0:
                phi.append(grab(last_phi))
 
            trange.set_postfix(acc=str(np.round(np.mean(np.array(accepted)), 3)))
 
    print(f"acceptance rate: {np.mean(accepted)}")
 
    
    phi = np.concatenate(phi)
    mag = jackknife(np.concatenate(mag))
    
    return {"phi" : phi,
            "accepted": accepted,
            "M" : mag}

In [8]:

def sample_MC(start, stop, step, model, batch_size, nr_batches, lam_cur, lattice_dim):
    kappas = np.arange(start, stop, step)
    chis = []
    mag = []
    acc_rate = []
    for kappa in kappas:

        action_cur = ScalarPhi4Action(kappa=kappa, lam=lam_cur)
        x_cond = torch.ones((batch_size, lattice_dim, lattice_dim)) * kappa * 10
        
        phi4_ens = record_flow_mc(model, action_cur, nr_batches, batch_size, 6, x_cond)
        
        x = phi4_ens["phi"]
        ind = np.where(np.sum(x, axis = (1,2)) < 0)
        x[ind] = x[ind] * -1
        
        chi = get_chi2(x)
        chis.append(chi)
        mag.append(phi4_ens["M"])
        acc_rate.append(np.mean(np.array(phi4_ens['accepted'])))
    
    return chis, mag, kappas, acc_rate

In [ ]:
chi_14, mag_14, kappa_plot, acc_rate = sample_MC(0.25, 0.28, 0.002, model, 64, 100, lam_cur, 14)